<h1 style="color:red;">Data Loading, Exploration and Preprocessing</h1>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from lightfm.data import Dataset
from lightfm.cross_validation import random_train_test_split
import optuna
from lightfm import LightFM
from lightfm.evaluation import precision_at_k, recall_at_k, auc_score
import joblib

In [ ]:
import numpy
import sklearn
import joblib

print("Numpy:", numpy.__version__)
print("scikit-learn:", sklearn.__version__)
print("joblib:", joblib.__version__)


In [ ]:
users = pd.read_csv('synthetic_users.csv') 
events = pd.read_csv('synthetic_events.csv') 
interactions = pd.read_csv('synthetic_interactions.csv')  

print("Users:")
print(users.head())
print("\nEvents:")
print(events.head())
print("\nInteractions:")
print(interactions.head())

In [ ]:
users.info()

In [ ]:
events.info()

In [ ]:
interactions.info()

<h2>Exploratory Data Analysis (EDA) - Users</h2>

In [ ]:
plt.figure(figsize=(14, 4))

plot_idx = 1

if 'gender' in users.columns:
    plt.subplot(1, 3, plot_idx)
    sns.countplot(data=users, x='gender')
    plt.title('Gender Distribution')
    plot_idx += 1

if 'age_group' in users.columns:
    plt.subplot(1, 3, plot_idx)
    sns.countplot(data=users, x='age_group')
    plt.title('Age Group Distribution')
    plot_idx += 1

if 'role' in users.columns:
    plt.subplot(1, 3, plot_idx)
    sns.countplot(data=users, x='role')
    plt.title('Role Distribution')
    plot_idx += 1

plt.tight_layout()
plt.show()


<h2>Exploratory Data Analysis (EDA) - Events</h2>

In [ ]:
print("Columns in events:", events.columns)
print("Is 'category' column present?", 'category' in events.columns)
print("Events DataFrame empty?", events.empty)

plt.figure(figsize=(10,4))

plt.subplot(1,2,1)
if 'category' in events.columns and not events.empty:
    sns.countplot(data=events, x='category')
    plt.title('Event Category Distribution')
    plt.xticks(rotation=45)
else:
    print("Skipping countplot: 'category' column missing or events is empty")

plt.subplot(1,2,2)
if not events.empty:
    sns.histplot(data=events, x='popularity_score', bins=20)
    plt.title('Popularity Score Distribution')
else:
    print("Skipping histplot: events DataFrame is empty")

plt.tight_layout()
plt.show()


<h2>Interaction Counts Analysis</h2>

In [ ]:
print("\nInteractions per user:")
if 'user_id' in interactions.columns:
    print(interactions['user_id'].value_counts().describe())
else:
    print(" 'user_id' column is missing in interactions.")

print("\nInteractions per event:")
if 'event_id' in interactions.columns:
    print(interactions['event_id'].value_counts().describe())
else:
    print("'event_id' column is missing in interactions.")


<h2>Removing uneeded data (users with role organizer )  </h2>

In [ ]:
users = users[users['role'] != 'organizer']
print(f"Users after removing organizers: {users.shape[0]}")

<h2>Remove interactions related to removed users</h2>

In [ ]:
valid_user_ids = set(users['user_id'])
interactions = interactions[interactions['user_id'].isin(valid_user_ids)]
print(f"Interactions after removing organizer-related ones: {interactions.shape[0]}")

<h2>Encode categorical columns</h2>

In [ ]:
users_encoded = users.copy()

for col in ['gender', 'age_group', 'role']:
    if col in users.columns:
        users_encoded[col] = users_encoded[col].astype('category').cat.codes
    else:
        print(f"Column '{col}' not found in users DataFrame.")

events_encoded = events.copy()  # this line is essential!

if 'category' in events.columns:
    events_encoded['category'] = events_encoded['category'].astype('category').cat.codes
else:
    print("Column 'category' not found in events DataFrame.")

print("\nEncoded Users sample:")
print(users_encoded.head())

print("\nEncoded Events sample:")
print(events_encoded.head())


<h2>Scale popularity_score</h2>

In [ ]:
scaler = MinMaxScaler()
events_encoded['popularity_score'] = scaler.fit_transform(events_encoded[['popularity_score']])

<h2>Convert event_date to datetime format</h2>

In [ ]:
events_encoded['event_date'] = pd.to_datetime(events_encoded['event_date'])

<h2>Final check: print samples and shapes after cleaning and encoding</h2>

In [ ]:
print("\nCleaned Users Sample:")
print(users_encoded.head())

print("\nCleaned Events Sample:")
print(events_encoded.head())

print("\nCleaned Interactions Sample:")
print(interactions.head())

<h1 style="color:red;">LightFM Dataset Preparation</h1>

<h2>Build interaction matrix from (user_id, event_id) pairs</h2> 

In [ ]:
dataset = Dataset()

user_ids = users_encoded['user_id'].unique()
event_ids = events_encoded['event_id'].unique()

user_features_raw = users_encoded.set_index('user_id')[['gender', 'age_group', 'role']]
user_feature_tuples = [
    (uid, [f"gender:{g}", f"age_group:{a}", f"role:{r}"])
    for uid, g, a, r in zip(user_features_raw.index, user_features_raw['gender'], user_features_raw['age_group'], user_features_raw['role'])
]

event_features_raw = events_encoded.set_index('event_id')[['category', 'popularity_score', 'is_weekend']]
event_feature_tuples = [
    (eid, [f"category:{c}", f"is_weekend:{w}"])
    for eid, c, w in zip(event_features_raw.index, event_features_raw['category'], event_features_raw['is_weekend'])
]

dataset.fit(
    users=user_ids,
    items=event_ids,
    user_features=set(f for _, feats in user_feature_tuples for f in feats),
    item_features=set(f for _, feats in event_feature_tuples for f in feats)
)

(interactions_matrix, weights) = dataset.build_interactions(
    [(row['user_id'], row['event_id']) for idx, row in interactions.iterrows()]
)
print(f"Interaction matrix shape: {interactions_matrix.shape}")

user_features = dataset.build_user_features(user_feature_tuples)
item_features = dataset.build_item_features(event_feature_tuples)

print("User features sparse:", user_features.shape)
print("Item features sparse:", item_features.shape)


<h1 style="color:red;">Training</h1>

<h2>Train-Test Split</h2>

In [ ]:
train, test = random_train_test_split(interactions_matrix, test_percentage=0.2, random_state=42)
print(f"Train interactions: {train.nnz} | Test interactions: {test.nnz}")

<h2>Hyperparameter Tuning with Optuna</h2>
<p>We use Optuna to automatically and efficiently find the best hyperparameters for the LightFM model. Optuna intelligently explores the parameter space by learning from previous trials, allowing faster convergence to optimal values compared to manual or random searches. This results in improved model performance with fewer training iterations and less computational effort.</p>

In [ ]:
def objective(trial):

    model = LightFM(
        no_components=trial.suggest_int("no_components", 20, 100),
        learning_rate=trial.suggest_float("learning_rate", 1e-4, 1e-1, log=True),
        item_alpha=trial.suggest_float("item_alpha", 1e-6, 1e-2, log=True),
        user_alpha=trial.suggest_float("user_alpha", 1e-6, 1e-2, log=True),
        loss="logistic",
        random_state=42
    )

 
    model.fit(train,
              user_features=user_features,
              item_features=item_features,
              epochs=20,
              num_threads=4,
              verbose=False)


    precision = precision_at_k(model, test,
                               user_features=user_features,
                               item_features=item_features,
                               k=5).mean()
    return precision


study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=20)

print("Best parameters:", study.best_params)
print("Best Precision@5:", study.best_value)

<h2> Train Final Model with Best Parameters</h2>

In [ ]:
best_params = study.best_params
model = LightFM(**best_params, random_state=42)

model.fit(train,
          user_features=user_features,
          item_features=item_features,
          epochs=20,
          num_threads=4)

<h1 style="color:red;">Evaluation</h1>
<ol>
    <li> <b>Precision@5: 0.0541</b>
This means that, on average, about 5.4% of the top 5 recommendations for a user are relevant or actually interacted with by the user. It reflects the accuracy of the top recommendations.</li>
    <li> <b>Recall@5: 0.1214</b>
This indicates that the system is able to retrieve around 12.1% of all relevant items (items the user interacted with) within the top 5 recommendations. It shows how well the system covers the user’s interests.</li>
    <li><b> AUC Score: 0.7430</b>
       
The Area Under the Curve (AUC) score of 0.743 means the model has a good overall ability to distinguish between items a user will interact with and those they will not. A score closer to 1 is better, so 0.743 shows solid predictive performance.

</li>
</ol>

In [ ]:
precision = precision_at_k(model, test,
                           user_features=user_features,
                           item_features=item_features,
                           k=5).mean()
recall = recall_at_k(model, test,
                     user_features=user_features,
                     item_features=item_features,
                     k=5).mean()
auc = auc_score(model, test,
                user_features=user_features,
                item_features=item_features).mean()

print(f"Precision@5: {precision:.4f}")
print(f"Recall@5: {recall:.4f}")
print(f"AUC Score: {auc:.4f}")


<h2>Generate Top-N Recommendations</h2>

In [ ]:
def generate_recommendations(model, user_ids, interactions_matrix, user_features, item_features, n=5):
    n_users, n_items = interactions_matrix.shape
    recommendations = {}

    for user_idx in range(n_users):
        scores = model.predict(user_ids=user_idx,
                               item_ids=np.arange(n_items),
                               user_features=user_features,
                               item_features=item_features)

        known_items = interactions_matrix.tocsr()[user_idx].indices
        scores[known_items] = -np.inf 

        top_items = np.argsort(-scores)[:n]
        recommendations[user_ids[user_idx]] = top_items

    return recommendations

<h2>Mapping & Displaying Recommendations (for user with id 0 )</h2> <p>this is used to have a better insight about the recommendation performance and the pattern followed</p>

In [ ]:
user_id_map, user_feature_map, item_id_map, item_feature_map = dataset.mapping()
inv_user_id_map = {v: k for k, v in user_id_map.items()}
real_user_id = inv_user_id_map[0]
print(f"Internal user ID 0 corresponds to user_id: {real_user_id}")
user_interactions = interactions[interactions['user_id'] == real_user_id]
print("Events user interacted with:")
print(user_interactions)
interacted_event_info = user_interactions.merge(events, on='event_id', how='left')
print(interacted_event_info)

<h2>Exploring Recommendation Diversity and Novelty</h2> <p>To evaluate the quality of our recommendation system beyond accuracy, we analyze how diverse (varied) and serendipitous (unexpected but relevant) the suggested items are. This helps ensure users receive a mix of familiar and novel content that keeps the experience engaging and personalized.</p>
<ol>
    <li>A lower value of Diversity means less variation in content types.
</li>
<li>A high Serendipity score means that many of the recommended items are new to the user and highly popular, indicating that the system is effectively introducing appealing and unexpected content.</li>
</ol>

In [ ]:
def get_recommendations_for_user(user_id, model, dataset, user_features, item_features, events_df, top_n=10):
    user_mapping = dataset.mapping()[0]
    event_mapping = dataset.mapping()[2] 
    inv_event_mapping = {v: k for k, v in event_mapping.items()}

    if user_id not in user_mapping:
        raise ValueError(f"User {user_id} not in dataset.")

    user_internal_id = user_mapping[user_id]

    scores = model.predict(user_internal_id, np.arange(len(event_mapping)), user_features=user_features, item_features=item_features)

    top_items_idx = np.argsort(-scores)[:top_n]
    
    recommended_event_ids = [inv_event_mapping[i] for i in top_items_idx]

    return recommended_event_ids, scores[top_items_idx]

In [ ]:
# Set the user ID to test
test_user_id = "cmdj139f30005gbika35k4jov"

# Call the function
try:
    recommended_ids, scores = get_recommendations_for_user(
        user_id=test_user_id,
        model=model,
        dataset=dataset,
        user_features=user_features,
        item_features=item_features,
        events_df=events_encoded,  
        top_n=5
    )

    print("Recommended Event IDs:", recommended_ids)
    print("Scores:", scores)

except ValueError as e:
    print("Error:", e)


In [ ]:

def diversity(recommendations, events_df):
    categories = events_df[events_df['event_id'].isin(recommendations)]['category']
    if len(categories) == 0:
        return 0
    return categories.nunique() / len(categories)

In [ ]:
def serendipity(recommendations, user_history, events_df):
    novel_items = set(recommendations) - set(user_history)
    if len(recommendations) == 0:
        return 0
    top_serendipitous = events_df[
        (events_df['event_id'].isin(novel_items)) & 
        (events_df['popularity_score'] > 0.9)
    ]
    return len(top_serendipitous) / len(recommendations)

In [ ]:
user_to_test = 'cmdj139f30005gbika35k4jov'

user_history = interactions[interactions['user_id'] == user_to_test]['event_id'].tolist()

recommended_events, rec_scores = get_recommendations_for_user(
    user_id=user_to_test,
    model=model,
    dataset=dataset,
    user_features=user_features,
    item_features=item_features,
    events_df=events_encoded,
    top_n=10
)

div_score = diversity(recommended_events, events_encoded)
ser_score = serendipity(recommended_events, user_history, events_encoded)

print(f"Recommendations for {user_to_test}: {recommended_events}")
print(f"Diversity: {div_score:.3f}")
print(f"Serendipity: {ser_score:.3f}")


<h2>Import the model</h2>

In [ ]:
import numpy
import joblib
import sklearn

print("NumPy version:", numpy.__version__)
print("Joblib version:", joblib.__version__)
print("Scikit-learn version:", sklearn.__version__)


In [ ]:
joblib.dump(model, 'recommendation_model.pkl')
joblib.dump(scaler, 'recommendation_scaler.pkl')
joblib.dump(user_features, 'recommendation_user_features.pkl')
joblib.dump(item_features, 'recommendation_item_features.pkl')
joblib.dump(dataset, 'recommendation_dataset.pkl')
